# 변수
+ 임의로 변할 수 있는 수
+ x라는 상자안에 데이터를 넣고 데이터는 변할 수 있음

In [84]:
import numpy as np

# variable 이라는 이름의 박스 설정
# data라는 변할 수 있는 수(데이터) 설정
class Variable:
    def __init__(self, data) -> None:
        self.data = data
        
    # # dezero에선 data가 numpy의 ndarray만 받기때문에 이를 확인하는 코드 추가
    # def __init__(self, data: np.ndarray) -> None:
    #     self.data = data

In [85]:
data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


차원 측정

In [86]:
x.data.ndim

0

In [87]:
np.zeros((2,)).ndim

1

In [88]:
np.zeros((2,3)).ndim

2

In [89]:
np.zeros((2,3,1)).ndim

3

# 함수
+ 입력값을 넣으면 함수에 저장된 수식에 따라 출력값을 냄
+ x -> function -> y

In [90]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x ** 2
        output = Variable(y)
        return output

In [91]:
f = Function()
x = Variable(np.array([[1,2],[3,4]]))
y = f(x)
y.data

array([[ 1,  4],
       [ 9, 16]])

In [92]:
# abstractmethod : 추상 메서드
from abc import abstractmethod

class Function:
    def __call__(self, input):
        x = input.data
        y = x ** 2
        output = Variable(y)
        return output
    
    
    # abstractmethod하고 raise NotImplementedError()가 똑같은 역할을 함.
    @abstractmethod
    def forward(self, x):
        pass
    
    def forward(self, x):
        raise NotImplementedError()

In [93]:
class Square(Function):
    def forward(self, x):
        return x**3

In [94]:
f = Square()
x = Variable(np.array([[2,1],[4,3]]))
y = f(x)
y.data

array([[ 4,  1],
       [16,  9]])

In [95]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [96]:
f = Exp()
x = Variable(np.array([[2,1],[4,3]]))
y = f(x)
y.data

array([[ 4,  1],
       [16,  9]])

In [97]:
square = Square()
exp = Exp()
x = Variable(np.array([[.2,.1],[.4,.3]]))
y = square(exp(square(x)))
y.data

array([[2.5600e-06, 1.0000e-08],
       [6.5536e-04, 6.5610e-05]])

# 수치 미분
+ 딥러닝에서 입력값을 계산하고 나온 loss값을 이용하여 weight를 update해야함.
+ 이때 weight를 업데이트 하기 위해 미분값이 필요함.
+ 오차가 적은 중앙차분을 이용하여 함수 설정

In [98]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

In [99]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
dy

4.000000000004

In [100]:
def ses(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

In [101]:
x = Variable(np.array(.2))
dy = numerical_diff(ses, x)
dy

0.00010240017920003061

# 역전파
+ 미분을 효율적으로 계산하여 수치 미분보다 계산 속도가 빠르고 정확도가 높다.
+ x -> 함수 -> ... -> 함수 -> y에서 dy/dx는 y부터 역순으로 미분한 값의 곱이 된다.
+ 이때, 순전파시 입력한 입력값이 필요하다.

# 함수 정의
이후 구현도 한번에 정의

In [102]:
import numpy as np

class Variable:
    def __init__(self, data) -> None:
        self.data = data
        self.grad = None
        self.creator = None
        
    # creator 설정 함수
    def set_creator(self, func):
        self.creator = func
        
    # 역전파 함수
    # 재귀적으로 구현
    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input_
            x.grad = f.backward(self.grad)
            x.backward()
           
    # 역전파 함수를 한번에 하기 위한 함수 
    # 재귀로 하는것보다 while문을 사용하여 반복문으로 구현하는게 더 효율적
    def backward_list(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input_, f.output_
            x.grad = f.backward(y.grad)
            if x.creator is not None:
                funcs.append(x.creator)
    
class Function:
    def __call__(self, input_):
        x = input_.data
        y = self.forward(x)
        output_ = Variable(y)
        output_.set_creator(self)
        self.input_ = input_
        self.output_ = output_
        return output_
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError() 
    
    
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        x = self.input_.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.input_.data
        gx = np.exp(x) * gy
        return gx

# 수동 순전파, 역전파

In [103]:
square1 = Square()
exp = Exp()
square2 = Square()

x = Variable(np.array(0.5))
a = square1(x)
b = exp(a)
y = square2(b)

y.grad = np.array(1.0)
b.grad = square2.backward(y.grad)
a.grad = exp.backward(b.grad)
x.grad = square1.backward(a.grad)
    
print(x.grad)

3.297442541400256


In [104]:
# x -> exp -> a -> sqaure1 -> y

x = Variable(np.array(0.5))
a = exp(x)
y = square1(a)

y.grad = np.array(1.0)
a.grad = square1.backward(y.grad)
x.grad = exp.backward(a.grad)

print(x.grad)

5.436563656918091


In [105]:
# x -> square1 -> a -> exp -> b -> exp -> c -> square2 -> y

x = Variable(np.array(0.5))
y = square2(exp(exp(square1(x))))

y.grad = np.array(1.0)
x.grad = square1.backward(exp.backward(exp.backward(square2.backward(y.grad))))

print(x.grad)

94.18146788281396


# creator
+ creator를 설정하여 변수와 함수의 동적 연결을 함.
+ 따라서 output을 통해 함수를 호출할 수 있고 이에 따라 역전파를 진행할 수 있게함

# 자동 역전파, Variable Creator 구현 테스트
+ define-by-run : 계산 그래프가 자동으로 생성되는 방식
+ define-and-run : tensorflow의 session처럼 설계를 하고 코드를 실행해야 되는 방식

In [106]:
square1 = Square()
exp = Exp()
square2 = Square()

x = Variable(np.array(0.5))
a = square1(x)
b = exp(a)
y = square2(b)

In [107]:
# y의 creator는 square2
assert y.creator == square2

In [108]:
# square2의 creator는 exp 이므로 y.creator가 아님
assert y.creator == exp

AssertionError: 

In [109]:
assert y.creator.input_ == b

In [110]:
assert y.creator.input_ == a

AssertionError: 

In [111]:
assert b.creator == exp

In [112]:
assert b.creator.input_ == a

In [114]:
assert y.creator.input_.creator.input_ == a

# 자동 역전파, creator 역전파 구현

In [115]:
y.grad = np.array(1.0)
square2 = y.creator
b = square2.input_
b.grad = square2.backward(y.grad)
b.grad

2.568050833375483

In [116]:
exp = b.creator
a = exp.input_
a.grad = exp.backward(b.grad)
a.grad

3.297442541400256

In [117]:
square1 = a.creator
x = square1.input_
x.grad = square1.backward(a.grad)
x.grad

3.297442541400256

# variable class안에 backward 추가하여 한번에 재귀적으로 모든 backward를 구하기

In [118]:
square1 = Square()
exp = Exp()
square2 = Square()

x = Variable(np.array(0.5))
y = square2(exp(square1(x)))

y.grad = np.array(1.0)

y.backward()
x.grad

3.297442541400256

In [119]:
square1 = Square()
exp = Exp()
square2 = Square()

x = Variable(np.array(0.5))
y = square2(exp(square1(x)))

y.grad = np.array(1.0)

y.backward_list()
x.grad

3.297442541400256

# 추가 개선사항
+ np.ones_like 사용
+ exp, square 클래스를 함수로 바로 사용할 수 있게 함수화

In [120]:
def exp(x):
    return Exp()(x)

def square(x):
    return Square()(x)

x = Variable(np.array(0.5))
y = square(exp(square(x)))

y.backward_list()
x.grad

3.297442541400256

---

+ 궁금한 내용 중간중간 코드로 해결
+ creator, input_으로 y에서 x값을 가져올수 있는데 의미 있는 것인가? -> 역전파도 순서대로 하기때문에 의미 없는것같다.